In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-04-13 10:02:51,400 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
gensim.__version__

'3.4.0'

In [4]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [5]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield text
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield text
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield text
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield text
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield text
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield text
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield text
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield text
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [6]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-04-13 10:03:14,080 : INFO : loading Dictionary object from stem100_dictionary
2018-04-13 10:03:14,098 : INFO : loaded stem100_dictionary


In [7]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-04-13 10:03:14,199 : INFO : loaded corpus index from stem100.mm.index
2018-04-13 10:03:14,199 : INFO : initializing cython corpus reader from stem100.mm
2018-04-13 10:03:14,200 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [8]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [9]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [10]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [12]:
%%time
model = gensim.models.fasttext.FastText(iter_token_corpus(), hs = 1, sg = 1, size=300, iter = 20, min_count=0, sample=.00001, workers=8)

2018-04-13 10:03:50,561 : INFO : collecting all words and their counts
2018-04-13 10:03:50,564 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-13 10:04:00,487 : INFO : PROGRESS: at sentence #10000, processed 11872635 words, keeping 28140 word types
2018-04-13 10:04:09,254 : INFO : PROGRESS: at sentence #20000, processed 22389436 words, keeping 31281 word types
2018-04-13 10:04:17,540 : INFO : PROGRESS: at sentence #30000, processed 32260628 words, keeping 32838 word types
2018-04-13 10:04:24,455 : INFO : PROGRESS: at sentence #40000, processed 40518216 words, keeping 34451 word types
2018-04-13 10:04:30,006 : INFO : PROGRESS: at sentence #50000, processed 47137590 words, keeping 35170 word types
2018-04-13 10:04:35,381 : INFO : PROGRESS: at sentence #60000, processed 53235450 words, keeping 36026 word types
2018-04-13 10:04:40,292 : INFO : PROGRESS: at sentence #70000, processed 58570547 words, keeping 36380 word types
2018-04-13 10:04:45,707 : INFO :

2018-04-13 10:08:03,512 : INFO : EPOCH 1 - PROGRESS: at 2.06% examples, 73822 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:04,532 : INFO : EPOCH 1 - PROGRESS: at 2.09% examples, 74010 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:05,545 : INFO : EPOCH 1 - PROGRESS: at 2.13% examples, 74292 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:06,559 : INFO : EPOCH 1 - PROGRESS: at 2.20% examples, 74445 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:07,574 : INFO : EPOCH 1 - PROGRESS: at 2.29% examples, 74425 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:08,642 : INFO : EPOCH 1 - PROGRESS: at 2.43% examples, 74306 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:09,660 : INFO : EPOCH 1 - PROGRESS: at 2.52% examples, 74373 words/s, in_qsize 0, out_qsize 1
2018-04-13 10:08:10,669 : INFO : EPOCH 1 - PROGRESS: at 2.59% examples, 74331 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:08:11,698 : INFO : EPOCH 1 - PROGRESS: at 2.66% examples, 74231 words/s, in_qsize 0, out_qsize 0
2

2018-04-13 10:09:19,143 : INFO : EPOCH 1 - PROGRESS: at 7.90% examples, 72967 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:20,157 : INFO : EPOCH 1 - PROGRESS: at 7.98% examples, 72984 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:21,171 : INFO : EPOCH 1 - PROGRESS: at 8.06% examples, 72966 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:22,220 : INFO : EPOCH 1 - PROGRESS: at 8.13% examples, 72942 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:23,256 : INFO : EPOCH 1 - PROGRESS: at 8.23% examples, 72906 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:24,260 : INFO : EPOCH 1 - PROGRESS: at 8.33% examples, 72896 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:25,290 : INFO : EPOCH 1 - PROGRESS: at 8.42% examples, 72850 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:26,288 : INFO : EPOCH 1 - PROGRESS: at 8.52% examples, 72873 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:09:27,319 : INFO : EPOCH 1 - PROGRESS: at 8.63% examples, 72811 words/s, in_qsize 0, out_qsize 1
2

2018-04-13 10:10:34,999 : INFO : EPOCH 1 - PROGRESS: at 15.64% examples, 73330 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:36,028 : INFO : EPOCH 1 - PROGRESS: at 15.77% examples, 73344 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:37,084 : INFO : EPOCH 1 - PROGRESS: at 15.89% examples, 73319 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:38,090 : INFO : EPOCH 1 - PROGRESS: at 15.98% examples, 73284 words/s, in_qsize 0, out_qsize 1
2018-04-13 10:10:39,158 : INFO : EPOCH 1 - PROGRESS: at 16.07% examples, 73266 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:40,202 : INFO : EPOCH 1 - PROGRESS: at 16.18% examples, 73269 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:41,254 : INFO : EPOCH 1 - PROGRESS: at 16.29% examples, 73263 words/s, in_qsize 0, out_qsize 1
2018-04-13 10:10:42,309 : INFO : EPOCH 1 - PROGRESS: at 16.39% examples, 73268 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:10:43,335 : INFO : EPOCH 1 - PROGRESS: at 16.49% examples, 73271 words/s, in_qsize 0, out_

2018-04-13 10:11:51,266 : INFO : EPOCH 1 - PROGRESS: at 24.35% examples, 73170 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:52,331 : INFO : EPOCH 1 - PROGRESS: at 24.44% examples, 73157 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:53,388 : INFO : EPOCH 1 - PROGRESS: at 24.53% examples, 73161 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:54,393 : INFO : EPOCH 1 - PROGRESS: at 24.63% examples, 73153 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:55,424 : INFO : EPOCH 1 - PROGRESS: at 24.75% examples, 73142 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:56,430 : INFO : EPOCH 1 - PROGRESS: at 24.90% examples, 73160 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:57,430 : INFO : EPOCH 1 - PROGRESS: at 25.00% examples, 73162 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:58,448 : INFO : EPOCH 1 - PROGRESS: at 25.12% examples, 73167 words/s, in_qsize 0, out_qsize 0
2018-04-13 10:11:59,515 : INFO : EPOCH 1 - PROGRESS: at 25.21% examples, 73168 words/s, in_qsize 0, out_

...

2018-04-13 15:18:02,933 : INFO : EPOCH 20 - PROGRESS: at 87.68% examples, 74807 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:03,969 : INFO : EPOCH 20 - PROGRESS: at 87.90% examples, 74808 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:04,985 : INFO : EPOCH 20 - PROGRESS: at 87.94% examples, 74809 words/s, in_qsize 5, out_qsize 0
2018-04-13 15:18:06,013 : INFO : EPOCH 20 - PROGRESS: at 88.02% examples, 74810 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:06,997 : INFO : EPOCH 20 - PROGRESS: at 88.11% examples, 74810 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:08,055 : INFO : EPOCH 20 - PROGRESS: at 88.20% examples, 74816 words/s, in_qsize 1, out_qsize 0
2018-04-13 15:18:09,084 : INFO : EPOCH 20 - PROGRESS: at 88.31% examples, 74824 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:10,099 : INFO : EPOCH 20 - PROGRESS: at 88.42% examples, 74822 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:18:11,132 : INFO : EPOCH 20 - PROGRESS: at 88.53% examples, 74831 words/s, in_qsiz

2018-04-13 15:19:17,615 : INFO : EPOCH 20 - PROGRESS: at 92.91% examples, 74921 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:18,646 : INFO : EPOCH 20 - PROGRESS: at 92.94% examples, 74918 words/s, in_qsize 0, out_qsize 1
2018-04-13 15:19:19,676 : INFO : EPOCH 20 - PROGRESS: at 92.97% examples, 74896 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:20,748 : INFO : EPOCH 20 - PROGRESS: at 93.00% examples, 74901 words/s, in_qsize 0, out_qsize 2
2018-04-13 15:19:21,752 : INFO : EPOCH 20 - PROGRESS: at 93.03% examples, 74913 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:22,767 : INFO : EPOCH 20 - PROGRESS: at 93.06% examples, 74919 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:23,786 : INFO : EPOCH 20 - PROGRESS: at 93.10% examples, 74913 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:24,794 : INFO : EPOCH 20 - PROGRESS: at 93.14% examples, 74928 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:19:25,819 : INFO : EPOCH 20 - PROGRESS: at 93.17% examples, 74923 words/s, in_qsiz

2018-04-13 15:20:32,556 : INFO : EPOCH 20 - PROGRESS: at 96.13% examples, 75169 words/s, in_qsize 0, out_qsize 1
2018-04-13 15:20:33,577 : INFO : EPOCH 20 - PROGRESS: at 96.19% examples, 75168 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:34,596 : INFO : EPOCH 20 - PROGRESS: at 96.25% examples, 75173 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:35,624 : INFO : EPOCH 20 - PROGRESS: at 96.31% examples, 75176 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:36,672 : INFO : EPOCH 20 - PROGRESS: at 96.38% examples, 75179 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:37,706 : INFO : EPOCH 20 - PROGRESS: at 96.45% examples, 75177 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:38,715 : INFO : EPOCH 20 - PROGRESS: at 96.51% examples, 75183 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:39,725 : INFO : EPOCH 20 - PROGRESS: at 96.57% examples, 75185 words/s, in_qsize 0, out_qsize 0
2018-04-13 15:20:40,730 : INFO : EPOCH 20 - PROGRESS: at 96.63% examples, 75190 words/s, in_qsiz

2018-04-13 15:21:42,588 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-13 15:21:42,635 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-13 15:21:42,672 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-13 15:21:42,673 : INFO : EPOCH - 20 : training on 259189466 raw words (71651868 effective words) took 953.4s, 75155 effective words/s
2018-04-13 15:21:42,674 : INFO : training on a 5183789320 raw words (1432828882 effective words) took 18845.8s, 76029 effective words/s


CPU times: user 1d 7h 46min 31s, sys: 6min 29s, total: 1d 7h 53min 1s
Wall time: 5h 17min 53s


In [13]:
model.save('FTsubsam')

2018-04-13 15:21:44,381 : INFO : saving FastText object under FTsubsam, separately None
2018-04-13 15:21:44,382 : INFO : storing np array 'vectors' to FTsubsam.wv.vectors.npy
2018-04-13 15:21:44,403 : INFO : not storing attribute buckets_word
2018-04-13 15:21:44,404 : INFO : storing np array 'vectors_ngrams' to FTsubsam.wv.vectors_ngrams.npy
2018-04-13 15:21:44,516 : INFO : not storing attribute vectors_norm
2018-04-13 15:21:44,517 : INFO : not storing attribute vectors_ngrams_norm
2018-04-13 15:21:44,517 : INFO : storing np array 'vectors_vocab' to FTsubsam.wv.vectors_vocab.npy
2018-04-13 15:21:44,540 : INFO : not storing attribute vectors_vocab_norm
2018-04-13 15:21:44,541 : INFO : storing np array 'syn1neg' to FTsubsam.trainables.syn1neg.npy
2018-04-13 15:21:44,564 : INFO : storing np array 'syn1' to FTsubsam.trainables.syn1.npy
2018-04-13 15:21:44,588 : INFO : storing np array 'vectors_ngrams_lockf' to FTsubsam.trainables.vectors_ngrams_lockf.npy
2018-04-13 15:21:44,702 : INFO : st

In [14]:
model = gensim.models.Word2Vec.load('FTsubsam')

2018-04-13 15:21:45,490 : INFO : loading Word2Vec object from FTsubsam
2018-04-13 15:21:45,802 : INFO : loading vocabulary recursively from FTsubsam.vocabulary.* with mmap=None
2018-04-13 15:21:45,802 : INFO : loading wv recursively from FTsubsam.wv.* with mmap=None
2018-04-13 15:21:45,803 : INFO : loading vectors from FTsubsam.wv.vectors.npy with mmap=None
2018-04-13 15:21:45,820 : INFO : loading vectors_ngrams from FTsubsam.wv.vectors_ngrams.npy with mmap=None
2018-04-13 15:21:45,913 : INFO : loading vectors_vocab from FTsubsam.wv.vectors_vocab.npy with mmap=None
2018-04-13 15:21:45,928 : INFO : setting ignored attribute buckets_word to None
2018-04-13 15:21:45,929 : INFO : setting ignored attribute vectors_norm to None
2018-04-13 15:21:45,930 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-04-13 15:21:45,930 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-04-13 15:21:45,931 : INFO : loading trainables recursively from FTsubsam.trainables.* with

In [ ]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

In [15]:
#preprocess a word
def T(word):
    if word not in en_stop:
        return ps.stem(word)

In [16]:
%%time

import math
from six import iteritems
from six.moves import xrange
idf = {}
for word, freq in dictionary.dfs.iteritems():
    idf[dictionary.id2token[word]] = math.log(dictionary.num_docs) - math.log(freq)


CPU times: user 52 ms, sys: 8 ms, total: 60 ms
Wall time: 42.5 ms


In [17]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [18]:
w2v = model.wv
def vec(document):
    return np.add.reduce([idf[i]*w2v[i] for i in document if i in dictionary.token2id])

In [20]:
%%time
doc_vec = []
for x in iter_token_corpus():
    x = vec(x)
    doc_vec.append(x)

CPU times: user 26min 11s, sys: 7.93 s, total: 26min 19s
Wall time: 26min 57s


In [21]:
from scipy.spatial.distance import cosine
def get_scores(query):
    scores = []
    i = 0
    query = vec(query)
    for x in doc_vec:
        if isinstance(x, np.ndarray):
            score = 1 - cosine(x, query)
        else:
            score = -1000
        scores.append(score)
    return scores

In [22]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        

CPU times: user 2min 19s, sys: 540 ms, total: 2min 20s
Wall time: 2min 19s


In [23]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.8873605728149414, 1853],
  ['ConsumerCourt_DCDRC_106530', '0', 0.8503319025039673, 11109],
  ['ConsumerCourt_DCDRC_107608', '1', 0.8924422264099121, 1336],
  ['ConsumerCourt_DCDRC_114291', '1', 0.9061210751533508, 442],
  ['ConsumerCourt_DCDRC_114382', '1', 0.8925884366035461, 1320],
  ['ConsumerCourt_DCDRC_118185', '1', 0.9079551100730896, 368],
  ['ConsumerCourt_DCDRC_130318', '0', 0.8982375264167786, 874],
  ['ConsumerCourt_DCDRC_130570', '1', 0.8954113125801086, 1083],
  ['ConsumerCourt_DCDRC_131146', '1', 0.9254036545753479, 30],
  ['ConsumerCourt_DCDRC_131717', '1', 0.9216412305831909, 59],
  ['ConsumerCourt_DCDRC_131741', '1', 0.9103593826293945, 292],
  ['ConsumerCourt_DCDRC_131818', '1', 0.91273033618927, 219],
  ['ConsumerCourt_DCDRC_131950', '0', 0.9088513255119324, 335],
  ['ConsumerCourt_DCDRC_131972', '0', 0.9260513782501221, 25],
  ['ConsumerCourt_DCDRC_132932', '0', 0.935602605342865, 2],
  ['ConsumerCourt_DCDRC_133592', '1',

In [24]:
model_name = 'FT_idf_subsam'
# Store results in 1st sheet
wb = Workbook()
dest_filename = model_name + '.xlsx'
ws1 = wb.active
ws1.title = " "
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)


#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [ ]:
x = f.parse(1)

In [25]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
f2 = pd.ExcelFile('all_results.xlsx')
append_df_to_excel("all_results.xlsx", f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel("all_results.xlsx", f.parse(2), sheet_name= f2.sheet_names[1])